<a href="https://colab.research.google.com/github/dattali18/IR_Assignments/blob/main/Assignment.03/IR_03_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IR Assignment 03

## Sentiment Analysis

In this assignment we are asked to perform sentiment analysis on the article we were given for assignment 01. This mean that in stage 1 we have built a corpus of positive (in our context) and negative words. In stage 2 we will use this corpus to perform sentiment analysis on the article.

> Install the required libraries

```bash
pip install tensorflow tensorflow_hub tensorflow_text transformers
```

In [1]:
!pip install tensorflow tensorflow_hub tensorflow_text transformers

In [ ]:
# import libraries

import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, BertTokenizer

In [4]:
import pandas as pd

# load the data
github_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.03/extracted_sentences.csv?raw=true"

df = pd.read_csv(github_link)

In [5]:
df.head()

,id,sentence,type
0,aj_1,pope renews call for gaza ceasefire release of...,pro-palestinian
1,aj_5,the hospital was raided by israeli forces befo...,pro-israeli
2,aj_10,the discovery of a mass grave inside the nasse...,pro-palestinian
3,aj_13,the propalestinian demonstrators were quickly ...,pro-palestinian
4,aj_18,thirteen out of 21 people killed by israel in ...,pro-israeli


In [6]:
# rename 'type' -> 'label'
df = df.rename(columns={'type': 'label'})

In [7]:
# label mapping
label_mapping = { 'pro-israeli': 0, 'pro-palestinian': 1, 'neutral': 2 }
df['label_int'] = df['label'].map(label_mapping)

# make sure all the labels are in the right format
# meaning if some labels didn't get mapped, then we should remove them

df = df.dropna(subset=['label_int'])

In [8]:
# get a subset of the data using random sampling
df_subset = df.sample(n=1000, random_state=1)

In [ ]:
from google.colab import auth
auth.authenticate_user()
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

In [ ]:
# split the data into training and testing 30% testing
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

X = df_subset['sentence']
y = df_subset['label_int']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# random_state is set to 42 so that the results are reproducible

# load the tokenizer
# take the smallest model possible since my machine is not very powerful
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_sentences(sentences, tokenizer, max_length):
    tokenized_sentences = tokenizer(
        list(sentences),
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )

    return   {
            "input_ids": tokenized_sentences["input_ids"],
            "attention_mask": tokenized_sentences["attention_mask"],
        }

# Tokenize the training and testing data
max_length = 128
X_train_tokenized = tokenize_sentences(X_train, tokenizer, max_length)
X_test_tokenized = tokenize_sentences(X_test, tokenizer, max_length)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [2]:
import tensorflow as tf

batch_size = 16
train_dataset = tf.data.Dataset.from_tensor_slices((dict(X_train_tokenized), y_train)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(X_test_tokenized), y_test)).batch(batch_size)

# load the model, get the smallest model possible
model_name = "bert-base-uncased"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=["accuracy"])

NameError: name 'X_train_tokenized' is not defined

In [ ]:
# train the model

EPOCHS = 5
history = model.fit(train_dataset, epochs=EPOCHS, validation_data=test_dataset, verbose=1)

# evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f"Test accuracy: {accuracy}")
print(f"Test loss: {loss}")

model_name_save = "model_1"
# save the model
model.save_pretrained(model_name_save)
tokenizer.save_pretrained(model_name_save)

In [ ]:
# plot the training history
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()